# 1. Radar
Por: Camila Lopes ([cclopes.me](https://cclopes.me/))  


Neste módulo, iremos aprender a lidar com dados de radares meteorológicos usando a linguagem Python, passando por leitura, visualização e processamento.


In [1]:
# Importando as bibliotecas necessárias
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## 1.1. Lendo dados de radar

### 1.1.1. Tipos de dados
Antes de aprender a ler dados de radar, é importante entender quais são os tipos de dados gerados por radares, já que eles podem ser configurados de diversas formas.

#### Escaneamento
A partir da fabricação do radar, é definido em qual banda ele irá operar e qual será sua configuração de escaneamento. Além de depender da banda de operação, essa configuração vai depender do local em que o radar será instalado e do objetivo dele (monitoramento ou pesquisa).  

Radares de monitoramento são de bandas de frequência menor (2 a 8 GHz, Bandas S e C) pois sofrem menos atenuação. Para tirar mais vantagem ainda disso, suas configurações de escaneamento são do tipo **volumétricas**, ou seja, em função do azimute (plano horizontal) e elevação (altura). A resolução temporal varia bastante dependendo principalmente da quantidade de elevações: dependendo do local a ser monitorado, uma quantidade menor ou maior de elevações é feita, com o objetivo final de ter uma resolução temporal razoável para monitoramento em tempo real (em geral essa resolução varia entre 5 e 15 minutos). Os dados gerados por esse tipo de configuração são no formato **PPI** (*Plan Position Indicator*), equivalente a uma matriz tridimensional mas não cartesiana por ser em função do azimute e elevação. Visualizações desses PPIs podem ser feitas no "plano horizontal" (imagem de todos os azimutes em uma elevação fixa) ou "vertical" (imagem de todas as elevações em um azimute fixo). Para gerar produtos mais simples de serem interpretados, o formato PPI é convertido para **CAPPI** (*Constant Altitude Plan Position Indicator*), agora sim equivalente a uma matriz tridimensional cartesiana (plano horizontal e altura). As visualizações dos CAPPIs podem ser feitas no plano horizontal (altura fixa) ou vertical (eixo x ou y fixo e altura variada).  

Radares de pesquisa costumam ser de frequência maior (acima de 8 GHz, Bandas X, K e W) pois, mesmo sofrendo mais atenuação principalmente de gotas de chuva, são mais sensíveis à gotículas de nuvem e cristais de gelo, permitindo estudos de microfísica de nuvens. Além do escaneamento volumétrico, uma configuração interessante para esses estudos é do tipo **setorial**, em que uma região de azimutes é fixada e o escaneamento é feito em função da elevação para cada azimute. Por medir apenas um setor, a resolução temporal pode ser menor que 5 minutos. Os dados gerados por esse tipo de configuração são no formato **RHI** (*Range Height Indicator*), equivalente a uma matriz tridimensional mas não cartesiana por ser em função da elevação e azimute. A visualização de RHIs em geral são feitas no "plano vertical" (distância em função da altura), mas também podem ser feitas no "plano horizontal" (todos os azimutes disponíveis em função de uma elevação).  

Os radares de pesquisa das bandas K e W (acima de 20 GHz) sofrem ainda mais atenuação, de forma que outra configuração de escaneamento que é usada (dependendo da fabricação do radar, é a única configuração disponível) é do tipo **apontamento vertical**, em que, como o nome já diz, a antena fica apontada para cima (ou para baixo, se estiver a bordo de um avião ou satélite), medindo uma mesma região ao longo do tempo. A resolução temporal nessa configuração é na escala de segundos a poucos minutos. Os dados gerados por esse tipo de configuração são no formato **THI** (*Time Height Indicator*), equivalente a uma matriz bidimensional em função da altura e tempo, com a visualização nesse mesmo plano.

#### Formato
Existem diferentes formatos de arquivos que armazenam os tipos de dados explicados acima. Os principais são:

- **Binário** (extensão `.bin` ou sem extensão): este é o formato mais simples de armazenamento. Ele pode ter diferentes tipos de estruturas (com cabeçalho ou não), mas não é possível saber essa estrutura apenas com o arquivo - é preciso ter algum README associado para conseguir abrir o arquivo. Sabendo essa estrutura, é possível abrir esse tipo de arquivo com a função `open` do Python. Como exemplo [baseado neste artigo do Towards Data Science](https://towardsdatascience.com/loading-binary-data-to-numpy-pandas-9caa03eb0672), supondo que a estrutura do arquivo `filename.bin` é simplesmente uma matriz de um nível de CAPPI de 500 x 500 pontos, em que cada ponto é um `integer` de 4 bytes, o código abaixo lê esse arquivo e converte para uma matriz do `numpy`:  

```
# Carregando a biblioteca
import numpy as np

# Definindo o formato da matriz
dt = np.dtype([('x', '<i4'), ('y', '<i4')])

# Abrindo o arquivo
with open("filename.bin", "rb") as f:
    binary_data = f.read()

# Convertendo para a matriz
np_data = np.frombuffer(binary_data, dt)
```

Esse tipo de arquivo ainda pode ser encontrado para dados de radar brutos, mas não para dados pré-processados ou processados por terem estruturas cada vez mais complexas e a necessidade de armazenar mais informações além dos dados em si; 

- **uf** (*Universal Format*, `.uf`): este formato foi desenvolvido nos anos 1970 e era o mais comum para armazenamento e distribuição de dados de radar. Ele foi desenvolvido para ser processado usando a linguagem FORTRAN e possui uma estrutura pré-definida com cabeçalho em caracteres ASCII e conteúdo em `integers` de 16 bits (saiba mais sobre ela no [documento do TITAN sobre o formato uf](https://ral.ucar.edu/projects/titan/docs/radial_formats/UfDoc.txt)). Como a estrutura é conhecida, é possível abrir esse tipo de arquivo no Python da mesma forma que dados binários, mas algumas bibliotecas como [`pyart`](https://arm-doe.github.io/pyart/index.html) possuem [funções que leem arquivos uf](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_uf.html?highlight=read%20uf);

- **HDF5** (*Hierarchical Data Format*, `.HDF5`): este formato é utilizado para armazenar grande quantidade de dados de quaisquer dimensões de forma mais organizada, com metadados associados. Diferente dos dados binários, não é preciso saber a estrutura dos dados. Em Python, é necessário usar a função `File()` da biblioteca [`h5py`](https://docs.h5py.org/en/stable/) para abrir esse tipo de arquivo. Algumas bibliotecas que processam dados no formato HDF5 usam essa biblioteca dentro de suas funções;

- **NetCDF** (*Network Common Data Form*, `.nc`): este formato foi desenvolvido nos anos 1980 pela [UCAR (*University Corporation for Atmospheric Research*)](https://www.ucar.edu/) especificamente para armazenar e compartilhar dados científicos. Atualmente na versão 4 [mantida pela Unidata](https://www.unidata.ucar.edu/software/netcdf/), ele possui suporte em Java, FORTRAN, C e diversas outras linguagens de programação. Similar ao HDF5, o formato NetCDF armazena dados em quaisquer dimensões com metadados. Existe uma convenção para metadados de meteorologia chamada [CF (*Climate and Forecast*)](http://cfconventions.org/) com o objetivo de padronizar e facilitar o processamento de dados de diversas fontes; para dados de radar, essa convenção é chamada [CfRadial](https://github.com/NCAR/CfRadial). Em Python, é possível abrir esse tipo de arquivo diretamente com a biblioteca [`netCDF4`](https://unidata.github.io/netcdf4-python/netCDF4/index.html), mas diversas bibliotecas usam essa como base para processar dados NetCDF, atualmente sendo recomendável a biblioteca [`xarray`](http://xarray.pydata.org/en/stable/weather-climate.html);

- Outros: outros formatos de arquivo estão associados aos softwares dos radares que geraram os dados. As bibliotecas [`pyart`](https://arm-doe.github.io/pyart/index.html) e [`wradlib`](https://docs.wradlib.org/en/stable/index.html) que veremos a seguir tem suporte a diversos formatos como [**MDV**](https://ral.ucar.edu/projects/titan/docs/MDV_XML_ICD.1.0.pdf), [**IRIS** (Vaisala)](ftp://ftp.sigmet.com/outgoing/manuals/), [**GAMIC**](https://www.gamic.com/), [**RAINBOW**](https://www.leonardocompany.com/en/products/rainbow-5-application-software), entre outros.  

Sabendo agora quais são os tipos de dados, podemos aprender como ler dados de radar com as bibliotecas Py-ART e wradlib.

### 1.1.2. Py-ART

A biblioteca Py-ART possui dois módulos que leem dados de radar: [`pyart.io`](https://arm-doe.github.io/pyart/API/generated/pyart.io.html) e [`pyart.aux_io`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.html). Os dois módulos possuem diversas funções que leem os diferentes tipos de dados explicados acima.

#### [`pyart.io.read()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read.html#pyart.io.read)
Essa é a função mais básica (*que deve ser usada primeiro*). Ela basicamente identifica qual o tipo de arquivo e lê usando a função correspondente (entre as opções [`read_cfradial()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_cfradial.html#pyart.io.read_cfradial), [`read_chl()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_chl.html#pyart.io.read_chl), [`read_mdv()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_mdv.html#pyart.io.read_mdv), [`read_nexrad_archive()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_archive.html#pyart.io.read_nexrad_archive), [`read_nexrad_cdm()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_cdm.html#pyart.io.read_nexrad_cdm), [`read_nexrad_level3()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_nexrad_level3.html#pyart.io.read_nexrad_level3), [`read_rsl()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_rsl.html#pyart.io.read_rsl), [`read_sigmet()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_sigmet.html#pyart.io.read_sigmet) e [`read_uf()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_uf.html#pyart.io.read_uf) disponíveis atualmente) de forma que você não precisa usar especificamente uma dessas funções. 

Como exemplo, o código abaixo lê um arquivo do radar de Presidente Prudente (SP) (do tipo Sigmet (IRIS)) e mostra as principais informações sobre ele. A forma mais rápida de verificar se o arquivo foi lido corretamente é usar a função [`[variável de radar].info()`](https://arm-doe.github.io/pyart/API/generated/pyart.core.Radar.info.html#pyart.core.Radar.info) para obter as informações dos dados. A saída dela mostra os atributos (variáveis, azimute, elevação, parâmetros do instrumento, metadados, etc) com o tipo de dado (`float`, `int`) e forma (quantidade de pontos e dimensões - por exemplo, a variável `reflectivity` tem 2 dimensões, 5760 x 957 pontos).

In [2]:
# Lendo o arquivo
dados_pp = pyart.io.read("../dados/radar/PP/PPR161225194610.RAWK318")
# Outra opção, se você já souber o formato do arquivo, é a usar a função correspondente
# dados_pp = pyart.io.read_sigmet("./dados/radar/PP/PPR161225194610.RAWK318")

# Informações sobre o arquivo
print(dados_pp.info(level="compact"))

altitude: <ndarray of type: float64 and shape: (1,)>
altitude_agl: None
antenna_transition: None
azimuth: <ndarray of type: float32 and shape: (5760,)>
elevation: <ndarray of type: float32 and shape: (5760,)>
fields:
	total_power: <ndarray of type: float32 and shape: (5760, 957)>
	reflectivity: <ndarray of type: float32 and shape: (5760, 957)>
	velocity: <ndarray of type: float32 and shape: (5760, 957)>
	spectrum_width: <ndarray of type: float32 and shape: (5760, 957)>
fixed_angle: <ndarray of type: float32 and shape: (16,)>
instrument_parameters:
	unambiguous_range: <ndarray of type: float32 and shape: (5760,)>
	prt_mode: <ndarray of type: |S4 and shape: (16,)>
	prt: <ndarray of type: float32 and shape: (5760,)>
	prt_ratio: <ndarray of type: float32 and shape: (5760,)>
	nyquist_velocity: <ndarray of type: float32 and shape: (5760,)>
	radar_beam_width_h: <ndarray of type: float32 and shape: (1,)>
	radar_beam_width_v: <ndarray of type: float32 and shape: (1,)>
	pulse_width: <ndarray of 

#### [`pyart.aux_io.read_[...]()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.html)

Se você tentar usar a função `pyart.io.read()` para abrir um arquivo e não funcionar, quer dizer que o arquivo tem um formato diferente dos suportados por essa função, de forma que o próximo passo é usar uma das funções do módulo `pyart.aux_io`. As opções disponíveis atualmente são: [`read_d3r_gcpex_nc()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_d3r_gcpex_nc.html#pyart.aux_io.read_d3r_gcpex_nc), [`read_edge_netcdf()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_edge_netcdf.html#pyart.aux_io.read_edge_netcdf), [`read_gamic()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_gamic.html#pyart.aux_io.read_gamic), [`read_kazr()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_kazr.html#pyart.aux_io.read_kazr), [`read_kazr_spectra()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_kazr_spectra.html#pyart.aux_io.read_kazr_spectra), [`read_noxp_iphex_nc()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_noxp_iphex_nc.html#pyart.aux_io.read_noxp_iphex_nc), [`read_odim_h5()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_odim_h5.html#pyart.aux_io.read_odim_h5), [`read_pattern()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_pattern.html#pyart.aux_io.read_pattern), [`read_radx()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_radx.html#pyart.aux_io.read_radx), [`read_rainbow_wrl()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_rainbow_wrl.html#pyart.aux_io.read_rainbow_wrl) e [`read_sinarame_h5()`](https://arm-doe.github.io/pyart/API/generated/pyart.aux_io.read_sinarame_h5.html#pyart.aux_io.read_sinarame_h5). Em um cenário ideal, você terá alguma informação sobre o formato do arquivo que ajude a escolher a função correta, mas como esse nem sempre é o caso, se acostume com a ideia de tentar usar todas e de que nenhuma delas pode funcionar (o que costuma acontecer com radares usados aqui no Brasil).

Como exemplos, o código abaixo lê arquivos de dois radares diferentes (São Roque - SP e SIPAM - AM) que tem o mesmo formato GAMIC-HDF5 e mostra suas informações.

In [3]:
# Lendo o arquivo
dados_sr = pyart.aux_io.read_gamic("../dados/radar/SR/SRO-250--2017-03-14--18-30-22.mvol")
dados_sipam = pyart.aux_io.read_gamic("../dados/radar/SIPAM/RADL08061720140103111200.HDF5")

# Informações sobre o arquivo
print("DADOS DE SÃO ROQUE")
print(dados_sr.info(level="compact"))
print("\nDADOS DO SIPAM")
print(dados_sipam.info(level="compact"))

DADOS DE SÃO ROQUE
altitude: <ndarray of type: float64 and shape: (1,)>
altitude_agl: None
antenna_transition: None
azimuth: <ndarray of type: float32 and shape: (5410,)>
elevation: <ndarray of type: float32 and shape: (5410,)>
fields:
	corrected_reflectivity: <ndarray of type: float32 and shape: (5410, 500)>
	reflectivity: <ndarray of type: float32 and shape: (5410, 500)>
	corrected_velocity: <ndarray of type: float32 and shape: (5410, 500)>
	spectrum_width: <ndarray of type: float32 and shape: (5410, 500)>
fixed_angle: <ndarray of type: float32 and shape: (15,)>
instrument_parameters:
	frequency: <ndarray of type: float32 and shape: (1,)>
	radar_beam_width_h: <ndarray of type: float32 and shape: (1,)>
	radar_beam_width_v: <ndarray of type: float32 and shape: (1,)>
	pulse_width: <ndarray of type: float32 and shape: (5410,)>
	prt: <ndarray of type: float32 and shape: (5410,)>
	prt_mode: <ndarray of type: <U9 and shape: (15,)>
	prt_ratio: <ndarray of type: float32 and shape: (5410,)>
	u

#### Outros (se nada der certo...)

Se nenhuma das opções acima funcionar, é hora de investigar. É difícil encontrar um formato de dados de radar que não esteja incluído em uma das funções citadas anteriormente, de forma que o que geralmente gera erro é diferenças nos nomes dos atributos ou variáveis. Isso acontece porque essas funções supõem que o arquivo tem uma estrutura com certos nomes pré-definidos (para radares americanos), que podem ser diferentes dos configurados nos radares brasileiros. Por exemplo, ao tentar um arquivo do radar de apontamento vertical MIRA (ATTO-Campina - AM) (que possui configuração similar ao radar KAZR do ARM) com a função correspondente (código abaixo), é gerado um erro `KeyError` associado ao atributo de latitude. Nesse caso, o atributo no arquivo do MIRA não é uma variável chamada `'lat'` (como esperado pelo comando `latitude = cfradial._ncvar_to_dict(ncvars['lat'])`) e sim uma subclasse do objeto (`ncobj.Latitude`).

In [4]:
pyart.aux_io.read_kazr("../dados/radar/MIRA_CAMPINA/20200306_0100.mmclx")

KeyError: 'lat'

A solução para corrigir esse problema é modificar a função de leitura, adaptando para esse formato dos dados. Nesse caso, eu precisei modificar vários outros trechos a ponto de criar uma nova função, chamada `read_mira()`, que está no arquivo `read_mira_radar.py`. O código abaixo usa essa função.

In [5]:
# Importando a função
from read_mira_radar import read_mira

# Lendo o arquivo
dados_mira = read_mira("../dados/radar/MIRA_CAMPINA/20200306_0100.mmclx")[0]

# Informações sobre o arquivo
dados_mira.metadata

/home/idies/miniconda3/envs/py37/lib/python3.7/site-packages/pywt/_utils.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/home/idies/workspace/Storage/camila_lopes/LabNuvens/tutoriais-python/1. Radar/read_mira_radar.py:225: RuntimeWarning: invalid value encountered in log10
  fields[field_name]["data"]


{'convention': 'CF-1.0',
 'location': 'ATTO Campina',
 'Altitude': '37m',
 'Latitude': '02.1814S',
 'Longitude': '59.0218W',
 'system': 'C5XC',
 'title': 'MIRA Cloud Radar Data',
 'institution': 'USP',
 'source': '200306_010004.pds.off',
 'reference': 'Ka Band Cloud Radar MIRA, METEK GmbH www.metek.de',
 'ppar': 'PRF:        5000\nPDR:         208\nSFT:         256\nAVC:         200\nIHP:           2\nCHG:         511\nPOL:         208\nATT:        1500\nTX:           0\nADCGAIN0:      0.00000\nADCGAIN1:      0.00000\nWND:           0\nPOS: -2147483424\nADD:          48\nLEN:           0\nCAL:          96\nNOS: -2147483648\nOF0:          42\nOF1:           0\nSWT:           0\nSUM:           0\nOSC:          26\nTST:           0\nCOR:           1\nOFS:           0\nHSBN:           0\nHSBA:           1\nCALIBRPOWER_M:      1.00000\nCALIBRSNR_M:      281.838\nCALIBRPOWER_S:      1.00000\nCALIBRSNR_S:      234.423\nRAW_GATE1:           2\nRAW_GATE2:         511\nRAW:           0\nPRC:    

Outros radares cujos dados não conseguem ser lidos pelas funções do Py-ART são o da FCTH (Salesópolis - SP) e o XPOL do Projeto CHUVA (já instalado em Manaus - AM, Campinas - SP e São Borja - RS). Os dados "brutos" deles estão no formato RAINBOW (separados por variável), que podem ser convertidos para HDF5-GAMIC com um [conversor desenvolvido pelo Dr. Thiago Biscaro (DSA/CPTEC/INPE)](https://github.com/tsbiscaro/Rainbow5_to_HDF5). Para abrir esses dados no formato GAMIC, é necessário uma versão modificada da `read_gamic()`, chamada `read_rainbow_hdf5()`, que está no arquivo `read_brazil_radar_py3.py`. O código abaixo lê e mostra informações dos arquivos dos dois radares.

In [6]:
# Importando a função
from read_brazil_radar_py3 import read_rainbow_hdf5

# Lendo os arquivos
dados_fcth = read_rainbow_hdf5("../dados/radar/FCTH/PNOVA2-20170314182730.HDF5")
dados_xpol = read_rainbow_hdf5("../dados/radar/XPOL_CMP/117BRX-20171115215006.HDF5")

# Informações sobre os arquivos
print("DADOS DO FCTH")
print(dados_fcth.info(level="compact"))
print("\nDADOS DO XPOL")
print(dados_xpol.info(level="compact"))

DADOS DO FCTH
altitude: <ndarray of type: float64 and shape: (1,)>
altitude_agl: None
antenna_transition: None
azimuth: <ndarray of type: float64 and shape: (2880,)>
elevation: <ndarray of type: float64 and shape: (2880,)>
fields:
	corrected_reflectivity: <ndarray of type: float64 and shape: (2880, 960)>
	reflectivity: <ndarray of type: float64 and shape: (2880, 960)>
	velocity: <ndarray of type: float64 and shape: (2880, 960)>
	spectrum_width: <ndarray of type: float64 and shape: (2880, 960)>
	differential_reflectivity: <ndarray of type: float64 and shape: (2880, 960)>
	filtered_differential_phase: <ndarray of type: float64 and shape: (2880, 960)>
	differential_phase: <ndarray of type: float64 and shape: (2880, 960)>
	specific_differential_phase: <ndarray of type: float64 and shape: (2880, 960)>
	cross_correlation_ratio: <ndarray of type: float64 and shape: (2880, 960)>
fixed_angle: <ndarray of type: float64 and shape: (8,)>
instrument_parameters:
	nyquist_velocity: <ndarray of type: 

Um último exemplo é de dados de CAPPIs do radar do SIPAM (AM) processados no período do GoAmazon (disponíveis no [repositório do ARM](https://adc.arm.gov/discovery/#/results/instrument_class_code::sipam-s-band-cappi)). O formato desses dados é similar aos dados em grade gerados pelo Py-ART, que são lidos com a função [`pyart.io.read_grid()`](https://arm-doe.github.io/pyart/API/generated/pyart.io.read_grid.htm). A função `read_sipam_cappi()` do arquivo `read_sipam_cappis.py` é uma adaptação dessa função original. O código abaixo lê o arquivo e mostra suas informações.

In [7]:
# Importando a função
from read_sipam_cappis import read_sipam_cappi

# Lendo o arquivo
dados_sipam_cappi = read_sipam_cappi("../dados/radar/SIPAM/sbmn_cappi_20140103_120016.nc")

# Informações sobre o arquivo
dados_sipam_cappi.metadata

{'Conventions': 'CF-1.0',
 'history': 'Applying azimuth offset: 0, time 2017/08/25 17:37:11\nApplying elevation offset: 0, time 2017/08/25 17:37:11\n',
 'institution': 'SIPAM/Texas A&M University',
 'references': 'http://atmo.tamu.edu/goamazon-sband-radar',
 'source': '',
 'title': 'unknown',
 'comment': 'v2.0a; Created at Texas A&M University (cschu@tamu.edu)'}

### 1.1.3. wrablib

## 1.2. Visualizando dados de radar

### 1.2.1. Py-ART

### 1.2.2. wradlib

## 1.3. Processando dados de radar

### 1.3.1. Py-ART

### 1.3.2. Py-ART + outras bibliotecas

### 1.3.3. wradlib